In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import gc
import re

import matplotlib.pyplot as plt
import seaborn as sns
import numpy.random as random
from pandas import Series, DataFrame

import lightgbm as lgb
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.tree import  DecisionTreeRegressor

#分布確認
!pip install ydata_profiling
import ydata_profiling as pdp

#前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder,OneHotEncoder,OrdinalEncoder
#モデリング
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold,StratifiedGroupKFold
from sklearn.metrics import f1_score,recall_score, precision_score
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
dfff_train = pd.read_csv("/sumple/train.csv")
dfff_test = pd.read_csv("/sumple/test.csv")

dff_train = dfff_train.copy()
dff_test = dfff_test.copy()

In [ ]:
#学習データとテストデータを一旦結合しておく
df_all = pd.concat([dff_train, dff_test], keys=["train","test"]).reset_index(level =0)

In [ ]:
#メモリ抑制
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f}MB".format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
             pass

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
#メモリ削減の実行
df_all = reduce_mem_usage(df_all)
df_train = reduce_mem_usage(df_all)
df_test = reduce_mem_usage(df_all)

Memory usage of dataframe is 1.06MB
Memory usage after optimization is: 0.73 MB
Decreased by 31.9%
Memory usage of dataframe is 0.73MB
Memory usage after optimization is: 0.73 MB
Decreased by 0.0%
Memory usage of dataframe is 0.73MB
Memory usage after optimization is: 0.73 MB
Decreased by 0.0%


In [ ]:
#共通の前処理を行うため、確認
df_all.head()

,level_0,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,customer_info,ProdTaken
0,train,0,50歳,Self Enquiry,2,900秒,Large Business,male,1.0,4.0,Basic,3.0,5,1,4,Executive,253905.0,未婚 車未所持 子供なし,1.0
1,train,1,56歳,Company Invited,1,14分,Salaried,Male,1.0,4.0,Standard,3.0,2,1,4,Senior Manager,404475.0,"離婚済み,車あり,子供無し",0.0
2,train,2,NaN,Self Enquiry,1,10分,Large Business,Female,1.0,3.0,Basic,3.0,4,0,4,Executive,278145.0,"結婚済み、自動車未所有,子供なし",1.0
3,train,3,三十七歳,Self Enquiry,2,1080秒,Small Business,female,1.0,3.0,Standard,4.0,1,0,5,Senior Manager,326805.0,離婚済み、車所持、子供無し,0.0
4,train,4,48歳,Company Invited,3,1020秒,Small Business,female,1.0,3.0,Basic,4.0,4,0,4,Executive,258435.0,独身／車所持／無子,1.0


In [ ]:
#目的変数確認
df_all["ProdTaken"].value_counts()

,count
ProdTaken,
0.0,2992
1.0,497


In [ ]:
#欠損値確認
df_all.isna().sum()

,0
level_0,0
id,0
Age,193
TypeofContact,18
CityTier,0
DurationOfPitch,252
Occupation,0
Gender,0
NumberOfPersonVisiting,0
NumberOfFollowups,57


In [ ]:
#前処理用に型変換
df_all["Age"] = df_all["Age"].astype(str)

In [ ]:
df_all["Age"].head()

,Age
0,50歳
1,56歳
2,nan
3,三十七歳
4,48歳


In [ ]:
# 全角数字を半角数字に変換するためのテーブルを作成
table = str.maketrans({chr(0xFF10 + i): chr(0x30 + i) for i in range(10)})

# NaN以外のデータを処理
for index, age in df_all["Age"].dropna().items():
    # age が数値型の場合は文字列に変換してから translate を適用
    if isinstance(age, (int, float)):
        normalized_age = str(age).translate(table)
    else:
        normalized_age = age.translate(table)
    df_all.loc[index, "Age"] = normalized_age
    df_all.loc[index, "Age_cleaned"] = normalized_age

In [ ]:
# 指定した文字を削除して表示
for index, age in df_all["Age_cleaned"].items():
    cleaned_age = age.replace('際', '').replace('歳', '').replace('才', '').replace('代', '')
    df_all.at[index, 'Age_cleaned'] = cleaned_age

df_all["Age_cleaned"].value_counts()

,count
Age_cleaned,
30,442
40,354
50,300
33,246
32,240
...,...
10,4
二十三,2
四十四,2


In [ ]:
#漢数字を変換
def convert_kanji_to_number(text):
    kanji_to_number = {
        "一": 1, "二": 2, "三": 3, "四": 4, "五": 5, "六": 6, "七": 7, "八": 8, "九": 9,
        "十": 10
    }
    result = 0
    temp_value = 0
    for char in text:
        if char in kanji_to_number:
            value = kanji_to_number[char]
            if value == 10:
                result += temp_value * 10
                temp_value = 0
            else:
                temp_value += value
    result += temp_value
    return result if result > 0 else text

# 適用前に "歳" を削除
df_all['Age_numeric'] = df_all['Age_cleaned'].str.replace("歳", "").apply(convert_kanji_to_number)

In [ ]:
df_all['Age_numeric'] = pd.to_numeric(df_all['Age_numeric'], errors='coerce')
df_all['Age_numeric'] = df_all['Age_numeric'].clip(lower=20, upper = 60)

In [ ]:
df_all["Age_numeric"].value_counts()

,count
Age_numeric,
30.0,454
40.0,360
50.0,306
33.0,272
32.0,252
36.0,246
34.0,244
37.0,240
35.0,234


In [ ]:
# 顧客への連絡方法:自分か企業からか
df_all["TypeofContact"].value_counts()

,count
TypeofContact,
Self Enquiry,4444
Company Invited,2516


In [ ]:
#都市層(1>2>3)
df_all["CityTier"].value_counts()

,count
CityTier,
2,3160
1,2991
3,827


In [ ]:
#営業担当者による顧客への売り込み時間
df_all["DurationOfPitch"].value_counts()

,count
DurationOfPitch,
8分,495
9分,488
15分,432
16分,408
14分,398
...,...
36分,10
1140秒,9
2160秒,5


In [ ]:
df_all["DurationOfPitch"] = df_all["DurationOfPitch"].astype(str)
# 新しい特徴量 "Pitch分" を作成
df_all["Pitch分"] = df_all["DurationOfPitch"].copy().values

In [ ]:
# 秒を削除
df_all['Pitch分'] = df_all['DurationOfPitch'].str.replace('秒', '')

# 数値に変換し、秒の場合は60で割る
df_all['Pitch_minutes'] = df_all['Pitch分'].apply(lambda x: int(x) / 60 if x.isdigit() else x)

In [ ]:
# 分を削除
df_all['Pitch_minutes'] = df_all['Pitch_minutes'].astype(str).str.replace('分', '')

df_all["Pitch_minutes"] = df_all["Pitch_minutes"].astype(float)
df_all['Pitch_minutes'] = df_all['Pitch_minutes'].astype('Int64')
df_all["Pitch_minutes"] = df_all["Pitch_minutes"].replace('nan', np.nan)

In [ ]:
#欠損値確認
df_all["Pitch_minutes"].isnull().sum()

252

In [ ]:
#顧客のご職業:中小、会社員、大企業
df_all["Occupation"].value_counts()

,count
Occupation,
Small Business,3456
Salaried,2801
Large Business,721


In [ ]:
df_all["Gender"].value_counts()

,count
Gender,
Male,2525
Female,1441
male,940
female,504
MALE,363
Ｍａｌｅ,260
Fe Male,213
FEMALE,197
Ｆｅｍａｌｅ,181


In [ ]:
df_all["Gender"] = df_all["Gender"].astype(str)
def convert_gender(gender):
  gender = gender.lower().replace(" ", "")  # 空白を削除して小文字に変換
  return gender  # 変換後の文字列を返す

df_all["Gender_enc"] = df_all["Gender"].apply(convert_gender)


In [ ]:
import unicodedata

df_all["Gender_enc"] = df_all["Gender_enc"].astype(str)

def convert_to_binary(gender):
    # 全角を半角に変換
    gender = unicodedata.normalize('NFKC', gender)
    # 全角英数字を半角英数字に変換
    gender = gender.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))

    # 小文字に変換
    gender = gender.lower()

    # 修正箇所: "female" または "f" を含んでいるかを確認
    if "female" in gender or "f" in gender:
        return 0
    else:
        return 1

df_all["Gender_enc"] = df_all["Gender_enc"].apply(convert_to_binary)

In [ ]:
display(df_all[["Gender", "Gender_enc"]].sample(n=20))
# femaleが1

,Gender,Gender_enc
247,Male,1
877,Male,1
1237,Female,0
2568,Female,0
263,Fe Male,0
2075,male,1
1516,male,1
2179,Male,1
1203,Fe Male,0
2605,Fe Male,0


In [ ]:
#一緒に旅行を予定している人数の合計
df_all["NumberOfPersonVisiting"].value_counts()

,count
NumberOfPersonVisiting,
2.0,3163
3.0,2773
1.0,686
4.0,356


In [ ]:
df_all["NumberOfFollowups"].isnull().sum()

57

In [ ]:
#顧客によるホテル施設の優先評価
df_all["PreferredPropertyStar"].value_counts()

,count
PreferredPropertyStar,
3.0,4328
4.0,2254
5.0,396


In [ ]:
#営業担当者による商品の売り込み
df_all["ProductPitched"].value_counts()

,count
ProductPitched,
Basic,1810
Deluxe,1660
Standard,1239
Super Deluxe,471
basic,218
...,...
Kıոg,1
Kiոg,1
Βasıc,1


In [ ]:
df_all["ProductPitched"] = df_all["ProductPitched"].astype(str)

def convert_gender_to_binary(product):
  # 全角を半角に変換し、空白を削除して小文字に変換
  product = unicodedata.normalize('NFKC', product).lower().replace(" ", "").replace("|", "l")

  return product

df_all["ProductPitched_low"] = df_all["ProductPitched"].apply(convert_gender_to_binary)

In [ ]:
def normalize_text(text):
  # 全角を半角に変換し、ギリシャ文字などを変換
  normalized_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8').lower()
  return normalized_text

df_all["ProductPitched_low"] = df_all["ProductPitched_low"].apply(normalize_text)

In [ ]:
def replace_multiple(text):
    # 正規表現によるbasic変換
    text = re.sub(r"^(ba?s?i?c?)+$", "basic", text)

    return text

df_all["ProductPitched_low"] = df_all["ProductPitched_low"].apply(replace_multiple)

In [ ]:
#basic残り
def replace_multiple(text):
    if text in ["asc", "asic", "asi"]:
        text = "basic"
    return text

df_all["ProductPitched_low"] = df_all["ProductPitched_low"].apply(replace_multiple)

In [ ]:
#king
def replace_multiple(text):
    if text in ["kig", "kng", "kg"]:
        text = "king"
    return text

df_all["ProductPitched_low"] = df_all["ProductPitched_low"].apply(replace_multiple)

In [ ]:
#deluxe
def replace_multiple(text):
    if text in ["eluxe","delue", "delux","dlux"]:
        text = "deluxe"
    return text

df_all["ProductPitched_low"] = df_all["ProductPitched_low"].apply(replace_multiple)

In [ ]:
#superdeluxe
def replace_multiple(text):
    if text in ["supereluxe","uperdeluxe", "superdluxe", "superdelue","supreluxe"]:
        text = "superdeluxe"
    return text

df_all["ProductPitched_low"] = df_all["ProductPitched_low"].apply(replace_multiple)

In [ ]:
#standard
def replace_multiple(text):
    if text in ["tandard","stadard","stndard","standar","stanard"]:
        text = "standard"
    return text

df_all["ProductPitched_low"] = df_all["ProductPitched_low"].apply(replace_multiple)

In [ ]:
df_all["ProductPitched_low"].value_counts()

,count
ProductPitched_low,
basic,2347
deluxe,2091
standard,1614
superdeluxe,640
king,286


In [ ]:
#顧客の年間旅行数
df_all["NumberOfTrips"].value_counts()

,count
NumberOfTrips,
2,1956
3,1461
5,836
1,669
7,445
年に2回,294
4,277
年に3回,245
6,234


In [ ]:
df_all["NumberOfTrips"] = df_all["NumberOfTrips"].astype(str)
def extract_trips(trips):
    if trips == "四半期に1回" or trips == "半年に1回":
        return trips  # "四半期に1回" と "半年に1回" はそのまま返す

    trips = trips.strip()  # trips の前後の空白や改行を削除

    original_trips = trips  # 元の値を保存

    # "年に" と "回" を削除
    trips = re.sub(r"年に?", "", trips)
    trips = re.sub(r"回?", "", trips)

    if trips.isdigit():  # trips が数字のみで構成されているか確認
        return int(trips)  # 数字のみであれば整数に変換して返す

    return original_trips  # それ以外の場合は元の値を返す

df_all["NumberOfTrips_year"] = df_all["NumberOfTrips"].apply(extract_trips)

In [ ]:
df_all["NumberOfTrips_year"].value_counts()

,count
NumberOfTrips_year,
2,2250
3,1706
5,977
1,801
7,513
4,327
6,281
nan,64
半年に1回,27


In [ ]:
#他のNumberを変換
df_all["NumberOfTrips_year"] = df_all["NumberOfTrips_year"].astype(str)
def replace_multiple(text):
    if text in ["半年に1回"]:
        text = 2
    elif text in ["四半期に1回"]:
        text = 4
    elif text == "nan":
        text = 0
    return text

df_all["NumberOfTrips_year"] = df_all["NumberOfTrips_year"].apply(replace_multiple)
df_all["NumberOfTrips_year"] = df_all["NumberOfTrips_year"].astype(int)
df_all["NumberOfTrips_year"].value_counts()

,count
NumberOfTrips_year,
2,2277
3,1706
5,977
1,801
7,513
4,333
6,281
0,64
8,26


In [ ]:
df_all["Passport"].value_counts()

,count
Passport,
0,6288
1,690


In [ ]:
#売り込みの満足度スコア
df_all["PitchSatisfactionScore"].value_counts()

,count
PitchSatisfactionScore,
2,2500
4,1452
1,1439
3,1344
5,243


In [ ]:
# 現在の組織における顧客の指定
df_all["Designation"].value_counts()

,count
Designation,
Executive,2219
Manager,2027
Senior Manager,1565
AVP,604
VP,259
...,...
Еxеcutivе,1
Еxecuｔive,1
E×ecｕtive,1


In [ ]:
df_all["Designation"] = df_all["Designation"].astype(str)

def convert_gender_to_binary(product):
  # 全角を半角に変換し、空白を削除して小文字に変換
  product = unicodedata.normalize('NFKC', product).lower().replace(" ", "").replace("|", "l")

  return product

df_all["Designation_low"] = df_all["Designation"].apply(convert_gender_to_binary)

In [ ]:
def normalize_text(text):
  # 全角を半角に変換し、ギリシャ文字などを変換
  normalized_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8').lower()
  return normalized_text

df_all["Designation_low"] = df_all["Designation_low"].apply(normalize_text)

In [ ]:
#executive
def replace_multiple(text):
    if text in ["excutive", "excutiv", "executiv","executie","xecutive",
                "eecutive", "excuti", "xecutiv", "executi", "cutive", "ecutive", "xcutiv","ecutie" ]:
        text = "executive"
    return text

df_all["Designation_low"] = df_all["Designation_low"].apply(replace_multiple)

In [ ]:
#manager
def replace_multiple(text):
    if text in ["anager", "managr", "manger", "mnager", "mnger", "nager",
                "mnagr", "mangr", "anagr", "nagr", "mngr" ]:
        text = "manager"
    return text

df_all["Designation_low"] = df_all["Designation_low"].apply(replace_multiple)

In [ ]:
#seniormanager
def replace_multiple(text):
    if text in ["seniormanagr", "sniormanager", "eniormanager", "seniormanger", "seniormnger", "enioranager",
                "sniormanagr", "seniormnager", "senioranager", "senioranger", "eniormanger" ]:
        text = "seniormanager"
    return text

df_all["Designation_low"] = df_all["Designation_low"].apply(replace_multiple)

In [ ]:
#vip
def replace_multiple(text):
    if text in ["avp", "vp" ]:
        text = "vip"
    return text

df_all["Designation_low"] = df_all["Designation_low"].apply(replace_multiple)

In [ ]:
df_all["Designation_low"].value_counts()

,count
Designation_low,
executive,2330
manager,2117
seniormanager,1632
vip,899


In [ ]:
#顧客の月収
df_all["MonthlyIncome"].head()

,MonthlyIncome
0,253905.0
1,404475.0
2,278145.0
3,326805.0
4,258435.0


In [ ]:
# df_all["MonthlyIncome"] をstr型に変換
df_all["MonthlyIncome"] = df_all["MonthlyIncome"].astype(str)

# 正規表現を使って数字を抽出
df_all["MonthlyIncome_ext"] = df_all["MonthlyIncome"].str.extract(r"月収(\d+\.?\d*)万円")

# 正規表現で抽出した値にのみ10000をかける
df_all["MonthlyIncome_ext"] = np.where(df_all["MonthlyIncome_ext"].notnull(), df_all["MonthlyIncome_ext"].astype(float) * 10000, df_all["MonthlyIncome"])

In [ ]:
df_all["MonthlyIncome"].head()

,MonthlyIncome
0,253905.0
1,404475.0
2,278145.0
3,326805.0
4,258435.0


In [ ]:
# 数値型に変換 ("nan" を NaN に変換)
df_all["MonthlyIncome_ext"] = pd.to_numeric(df_all["MonthlyIncome_ext"], errors='coerce')

In [ ]:
#顧客の情報のメモ(婚姻状況や車の有無、旅行の子どもの同伴の有無について記載されている)
df_all["customer_info"].value_counts()

,count
customer_info,
結婚済み、車未所持、子供1人,74
結婚済み、車所持、子供1人,57
未婚、車未所持、子供1人,41
結婚済み 車未所持 子供1人,41
結婚済み、車所持、子供2人,38
...,...
未婚/車なし/子供2人,1
未婚、車なし、2児,1
未婚/車所持／子供無し,1


In [ ]:
df_all["child_info"] = df_all["customer_info"].str.replace(r"[\s()]", "", regex=True)
child_info_temp1 = df_all["child_info"].str.extract(r"(子供.+|こども.+)")

child_info_temp3 = child_info_temp1.copy()
child_info_temp2 = df_all["customer_info"].str.extract(r"(1児|2児|3児|無子|不明|わからない|子育て状況不明|子の数不詳|非育児家庭)\s?")
df_all["child_info"] = child_info_temp3[0].fillna(child_info_temp2[0])

In [ ]:
# "customer_info" から  "customer_info_removed" を作成
df_all["customer_info_removed"] = df_all["customer_info"].str.replace(r"(子供|こども|1児|2児|3児|非育児家庭|無子|不明|わからない|子育て状況不明|子の数不詳).*\s?", "", regex=True)

In [ ]:
df_all["child_info"].value_counts()
df_all["child_info"].isna().sum()

0

In [ ]:
display(df_all[["customer_info", "customer_info_removed","child_info"]].sample(n=10))

,customer_info,customer_info_removed,child_info
1105,結婚済み\t車所持\tこども2人,結婚済み\t車所持\t,こども2人
3234,結婚済み、自動車所有、子供2人,結婚済み、自動車所有、,子供2人
3094,結婚済み/車未所持/子供1人,結婚済み/車未所持/,子供1人
3349,結婚済み\t自動車未所有\t子供1人,結婚済み\t自動車未所有\t,子供1人
1015,結婚済み\t車未所持\t子供1人,結婚済み\t車未所持\t,子供1人
3091,独身 車未所持 子供1人,独身 車未所持,子供1人
3203,結婚済み 車所持 子供有り 2人,結婚済み 車所持,子供有り2人
1637,"離婚済み,自動車未所有,こども1人","離婚済み,自動車未所有,",こども1人
2497,"結婚済み,車未所持,こども2人","結婚済み,車未所持,",こども2人
1391,"結婚済み、車所持,子供1人","結婚済み、車所持,",子供1人


In [ ]:
df_all["customer_info_removed"].value_counts()

,count
customer_info_removed,
結婚済み、車未所持、,248
結婚済み、車所持、,222
離婚済み、車所持、,160
結婚済み 車所持,159
結婚済み 車未所持,144
...,...
離婚済み／自家用車なし/,1
"未婚,自動車所有、",1
"独身、車なし,",1


In [ ]:
df_all["material_status"] = df_all["customer_info_removed"].str.extract(r"(結婚済み|未婚|離婚済み|独身|)")

In [ ]:
df_all["customer_info_rem2"] = df_all["customer_info_removed"].str.replace(r"(結婚済み|未婚|離婚済み|独身)", "", regex=True)

In [ ]:
df_all["material_status"].value_counts()

,count
material_status,
結婚済み,2846
離婚済み,1688
独身,1309
未婚,1135


In [ ]:
df_all["customer_info_rem2"].value_counts()

,count
customer_info_rem2,
、車未所持、,604
、車所持、,559
車未所持,387
車所持,362
、自動車未所有、,303
...,...
/車保有なし／,1
",車保有、",1
／車あり/,1


In [ ]:
df_all["car_status"] = df_all["customer_info_removed"].str.extract(r"(所持|未所持|未所有|所有|なし|あり|保有なし|保有)")

In [ ]:
df_all["car_status"].isna().sum()

0

In [ ]:
display(df_all[["customer_info", "child_info","material_status", "car_status"]].sample(n=10))

,customer_info,child_info,material_status,car_status
1205,離婚済み\t自家用車あり\t子供無し,子供無し,離婚済み,あり
1959,結婚済み、乗用車なし、子供1人,子供1人,結婚済み,なし
1397,結婚済み、自家用車あり、子供1人,子供1人,結婚済み,あり
2063,離婚済み／自動車所有／子供1人,子供1人,離婚済み,所有
3081,"結婚済み,自家用車あり,子供1人",子供1人,結婚済み,あり
15,独身 車所持 子供なし,子供なし,独身,所持
2637,未婚 自動車未所有 こども2人,こども2人,未婚,未所有
1706,結婚済み、車未所持、こども1人,こども1人,結婚済み,未所持
460,結婚済み/乗用車なし/子供有り 1人,子供有り1人,結婚済み,なし
871,独身、車未所持、こども1人,こども1人,独身,未所持


In [ ]:
df_all["child_info"].value_counts()

,count
child_info,
子供1人,1699
子供なし,1038
子供2人,921
こども1人,832
子供無し,507
こども2人,456
子供有り1人,383
子供有り2人,236
1児,220


In [ ]:
# child_info_int 列をコピー
df_all['child_info_int'] = df_all['child_info'].copy()

# 数値に変換できる値を整数に変換
df_all['child_info_int'] = pd.to_numeric(df_all['child_info_int'].str.extract(r'(\d+)')[0], errors='coerce')

# 整数に変換できなかった場合は元の値を保持
df_all['child_info_int'] = df_all['child_info_int'].fillna(df_all['child_info'])

In [ ]:
df_all["child_info_cat"] =df_all["child_info_int"].copy()

In [ ]:
df_all["child_info_int"].value_counts()

,count
child_info_int,
1.0,3134
2.0,1721
子供なし,1038
子供無し,507
3.0,140
子供ゼロ,125
無子,121
非育児家庭,118
子供の数不明,39


In [ ]:
def convert_child_info(value):
  if value in ["子供なし", "子供無し","子供ゼロ", "無子", "非育児家庭","子供の数不明", "不明","子育て状況不明", "子の数不詳", "わからない"]:
    return 0
  else:
    return value

df_all["child_info_int"] = df_all["child_info_int"].apply(convert_child_info)

In [ ]:
df_all["child_info_int"].value_counts()

,count
child_info_int,
1.0,3134
0.0,1983
2.0,1721
3.0,140


In [ ]:
def convert_child_info1(value):
  if value in ["子供なし", "子供無し","子供ゼロ", "無子", "非育児家庭"]:
    return 0
  elif value in ["子供の数不明", "不明","子育て状況不明", "子の数不詳", "わからない"]:
    return -1
  else:
    return value

df_all["child_info_cat"] = df_all["child_info_cat"].apply(convert_child_info1)

In [ ]:
df_all["child_info_cat"].value_counts()

,count
child_info_cat,
1.0,3134
0.0,1909
2.0,1721
3.0,140
-1.0,74


In [ ]:
# #離婚済は経済環境に直結していると考えて残しておく
df_all['material_count'] = df_all['material_status'].copy()

def convert_material_info(value):
  if value in ["結婚済み"]:
    return 2
  elif value in ["離婚済み"]:
    return 1
  elif value in ["独身", "未婚"]:
    return 0
  else:
    return value

df_all["material_count"] = df_all["material_count"].apply(convert_material_info)

In [ ]:
df_all["material_status"].value_counts()

,count
material_status,
結婚済み,2846
離婚済み,1688
独身,1309
未婚,1135


In [ ]:
df_all["material_count"].value_counts()

,count
material_count,
2,2846
0,2444
1,1688


In [ ]:
df_all["car_status"].value_counts()

,count
car_status,
所持,1787
未所持,1674
未所有,887
なし,856
所有,789
あり,583
保有なし,202
保有,200


In [ ]:
# 元の特徴量も残しておく
df_all['car_status_count'] = df_all['car_status'].copy()

def convert_car_info(value):
  if value in ["所持","所有", "あり", "保有"]:
    return 1
  elif value in ["未所持", "未所有", "なし","保有なし", ""]:
    return 0
  else:
    return value

df_all["car_status_count"] = df_all["car_status_count"].apply(convert_car_info)

In [ ]:
df_all["car_status_count"].value_counts()

,count
car_status_count,
0,3619
1,3359


In [ ]:
df_all.isnull().sum()

,0
level_0,0
id,0
Age,0
TypeofContact,18
CityTier,0
DurationOfPitch,0
Occupation,0
Gender,0
NumberOfPersonVisiting,0
NumberOfFollowups,57


In [ ]:
#型の確認
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6978 entries, 0 to 3488
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   level_0                 6978 non-null   object 
 1   id                      6978 non-null   int16  
 2   Age                     6978 non-null   object 
 3   TypeofContact           6960 non-null   object 
 4   CityTier                6978 non-null   int8   
 5   DurationOfPitch         6978 non-null   object 
 6   Occupation              6978 non-null   object 
 7   Gender                  6978 non-null   object 
 8   NumberOfPersonVisiting  6978 non-null   float16
 9   NumberOfFollowups       6921 non-null   float16
 10  ProductPitched          6978 non-null   object 
 11  PreferredPropertyStar   6978 non-null   float16
 12  NumberOfTrips           6978 non-null   object 
 13  Passport                6978 non-null   int8   
 14  PitchSatisfactionScore  6978 non-null   int8 

In [ ]:
#型変換の微調整
df_all["Passport"] = df_all["Passport"].astype(int)
df_all["PitchSatisfactionScore"] = df_all["PitchSatisfactionScore"].astype(int)

In [ ]:
for col in df_all.columns:
  if df_all[col].dtype == 'float16':
    df_all[col] = df_all[col].astype('float64')

In [ ]:
for col in df_all.columns:
  if df_all[col].dtype == "int8":
    df_all[col] = df_all[col].astype("int64")

In [ ]:
for col in df_all.columns:
  if df_all[col].dtype == "object":
    df_all[col] = df_all[col].astype("category")

In [ ]:
for col in df_all.columns:
  if df_all[col].dtype == "int16":
    df_all[col] = df_all[col].astype("int64")

In [ ]:
#ここまで共通の前処理
df_all0 = df_all.copy()